#### Problem 3

In [51]:
from itertools import product

def purchase_by_user(threshold, emoticons, discounts):
    purchase = 0
    for emoticon, discount in zip(emoticons, discounts):
        if discount >= threshold:
            purchase += emoticon * (1 - discount)
    
    return int(purchase)

def solution(users, emoticons):
    discount_list = [0.1, 0.2, 0.3, 0.4]
    
    result = []
    for discounts in product(discount_list, repeat=len(emoticons)):
        emoticon_plus = 0
        profit = 0
        for user in users:
            purchase = purchase_by_user(
                threshold=user[0]/100, 
                emoticons=emoticons, 
                discounts=discounts
            )
            
            if purchase >= user[1]:
                emoticon_plus += 1
            else:
                profit += purchase
        
        result.append([emoticon_plus, profit])
    
    result = sorted(result, key=lambda x: (-x[0], -x[1]))
    
    return result[0]

In [52]:
users1 = [[40, 10000], [25, 10000]]
emoticons1 = [7000, 9000]

users2 = [[40, 2900], [23, 10000], [11, 5200], [5, 5900], [40, 3100], [27, 9200], [32, 6900]]
emoticons2 = [1300, 1500, 1600, 4900]

In [53]:
solution(users1, emoticons1)

[1, 5400]

In [54]:
solution(users2, emoticons2)

[4, 13860]

#### Problem 5

In [140]:
def update(arr, cmd, loc, merge_map):
    if len(cmd) == 4:
        r, c = int(cmd[1]), int(cmd[2])
        prev = arr[r][c]
        word = cmd[3]
        if merge_map.get((r, c), False):
            for i, j in merge_map[(r, c)]:
                if prev != 'EMPTY':
                    loc[prev].remove((i, j))
                if word in loc:
                    loc[word].append((i, j))
                else:
                    loc[word] = [(i, j)]
                arr[i][j] = word

        else:
            if prev != 'EMPTY':
                loc[prev].add((r, c))
            if word in loc:
                loc[word].add((r, c))
            else:
                loc[word] = set([(r, c)])
            arr[r][c] = word

    else:
        word1, word2 = cmd[1], cmd[2]
        loc_list = loc[word1]
        for r, c in loc_list:
            arr[r][c] = word2

        if loc.get(word2, False):
            loc[word2] = loc[word2] + loc_list
        else:
            loc[word2] = loc_list

        del loc[word1]

    return arr, loc


def merge(arr, cmd, loc, merge_map):
    r1, c1, r2, c2 = map(int, cmd[1:])
    if arr[r1][c1] != 'EMPTY':
        word = arr[r1][c1]
    else:
        word = arr[r2][c2]

    merge_set = set()
    for i in range(min(r1, r2), max(r1, r2) + 1):
        for j in range(min(c1, c2), max(c1, c2) + 1):
            prev = arr[i][j]
            if loc.get(prev, False):
                loc[prev].remove((i, j))

            arr[i][j] = word

            merge_set.add((i, j))
            if merge_map.get((i, j), False):
                merge_set.update(merge_map[(i, j)])

    for i, j in merge_set:
        if (i, j) in merge_map:
            merge_map[(i, j)].update(merge_set)
        else:
            merge_map[(i, j)] = merge_set

    loc[word] = loc[word] | merge_set

    return arr, loc, merge_map


def unmerge(arr, cmd, loc, merge_map):
    r, c = map(int, cmd[1:])
    merge_list = merge_map[(r, c)]
    for i, j in merge_list:
        del merge_map[(i, j)]
        if i == r and j == c:
            continue
        else:
            prev = arr[i][j]
            if loc.get(prev, False):
                loc[prev].remove((i, j))

            arr[i][j] = 'EMPTY'

    return arr, loc, merge_map


def solution(commands):
    arr = [['EMPTY'] * 51 for _ in range(51)]

    result = []
    loc = {}
    merge_map = {}
    for cmd in commands:
        cmd = cmd.split(' ')
        if cmd[0] == 'UPDATE':
            arr, loc = update(arr, cmd, loc, merge_map)
        elif cmd[0] == 'MERGE':
            arr, loc, merge_map = merge(arr, cmd, loc, merge_map)
        elif cmd[0] == 'UNMERGE':
            arr, loc, merge_map = unmerge(arr, cmd, loc, merge_map)
        elif cmd[0] == 'PRINT':
            result.append(arr[int(cmd[1])][int(cmd[2])])

    return result

In [141]:
commands1 = [
    "UPDATE 1 1 menu", 
    "UPDATE 1 2 category", 
    "UPDATE 2 1 bibimbap", 
    "UPDATE 2 2 korean", 
    "UPDATE 2 3 rice", 
    "UPDATE 3 1 ramyeon", 
    "UPDATE 3 2 korean", 
    "UPDATE 3 3 noodle", 
    "UPDATE 3 4 instant",
    "UPDATE 4 1 pasta", 
    "UPDATE 4 2 italian", 
    "UPDATE 4 3 noodle", 
    "MERGE 1 2 1 3", 
    "MERGE 1 3 1 4", 
    "UPDATE korean hansik", 
    "UPDATE 1 3 group", 
    "UNMERGE 1 4", 
    "PRINT 1 3", 
    "PRINT 1 4"
]

commands2 = [
    "UPDATE 1 1 a", "UPDATE 1 2 b", "UPDATE 2 1 c", "UPDATE 2 2 d", "MERGE 1 1 1 2", 
    "MERGE 2 2 2 1", "MERGE 2 1 1 1", "PRINT 1 1", "UNMERGE 2 2", "PRINT 1 1"
]

commands3 = [
    "UPDATE 1 1 a",
    "UPDATE 2 2 b",
    "UPDATE 3 3 c",
    "MERGE 1 1 2 2",
    "MERGE 2 2 3 3",
    "UNMERGE 2 2",
]

In [146]:
solution(commands2)

['d', 'EMPTY']

#### Problem 7

In [168]:
from collections import deque


def tree(graph, direction):
    queue = deque([1])

    while queue:
        node = queue.popleft()
        if direction.get(node, False):
            queue.append(direction[node])
            graph, direction = update_direction(graph, direction, node)

    return graph, node, direction


def update_direction(graph, direction, node):
    next_node = graph[node].popleft()
    direction[node] = next_node
    graph[node].append(next_node)

    return graph, direction


def solution(edges, target):
    graph = {}
    for u, v in edges:
        if u in graph:
            graph[u].append(v)
        else:
            graph[u] = [v]

    for u, v_list in graph.items():
        graph[u] = deque(sorted(v_list))

    direction = {}
    for node, edge in graph.items():
        direction[node] = min(edge)

    seq = []
    max_len = sum(target)
    i = 0
    while i < max_len:
        i += 1
        graph, loc, direction = tree(graph, direction)
        seq.append(loc)

    return seq

In [169]:
edges1 = [[2, 4], [1, 2], [6, 8], [1, 3], [5, 7], [2, 5], [3, 6], [6, 10], [6, 9]]
target1 = [0, 0, 0, 3, 0, 0, 5, 1, 2, 3]

In [170]:
print(solution(edges1, target1))

[4, 8, 7, 9, 4, 10, 7, 8, 4, 9, 7, 10, 4, 8]


In [171]:
sum(target1)

14

#### Problem 6

In [ ]:
def solution(n, m, x, y, r, c, k):
    answer = ''
    cnts = {'d':0, 'l':0, 'r':0, 'u':0}
    row = x - r
    col = y - c
    cnt = abs(row) + abs(col)
    k -= cnt
    if k % 2 == 1:
        answer = 'impossible'
    else:
        if row > 0:
            cnts['u'] = row
        else:
            cnts['d'] = -row

        if col > 0:
            cnts['l'] = col
        else:
            cnts['r'] = -col

        answer = 'd'*cnts['d']
        last = 'u'*cnts['u']
        if r < n:
            move = min(k//2, n-r)
            answer += 'd' * move
            last += 'u' * move
            k -= 2*move

        answer += 'l'*cnts['l']
        last = 'r'*cnts['r'] + last
        if c > 1:
            move  = min(k // 2 , c-2)
            answer += 'l' * move
            last = 'r' * move + last
            k -= 2*move
        else:
            if k >= 2:
                answer += 'r'
                last = 'l' + last
                k -= 2
        answer += 'lr' * (k//2) + last
    return answer
